In [ ]:
import pandas as pd
data = pd.read_json("train.jsonl", lines=True) #on importe le fichier .jsonl

In [ ]:
import nltk
nltk.download('stopwords')
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 69.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#### On procède au prétraitement
stop_words = set(stopwords.words('french'))
nlp = spacy.load("fr_core_news_sm")

# On charge les données
data = pd.read_json("train.jsonl", lines=True)

# Étape 1 : Nettoyage du texte
def clean_text(text):
    text = text.lower()  # Mise en minuscules
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # caractères spéciaux et des chiffres supprimés
    text = re.sub(r'\s+', ' ', text).strip()  # espaces en trop supprimés
    return text

# Étape 2 : Lemmatisation
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]
    return " ".join(lemmatized_words)

# Appliquer le nettoyage et la lemmatisation
data['cleaned_text'] = data['texte_annonce'].apply(clean_text).apply(lemmatize_text)

# Étape 3 : Vectorisation avec TF-IDF
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=0.8)
X_tfidf = tfidf_vectorizer.fit_transform(data['cleaned_text'])

# Étape 4 : Réduction de dimension
# Truncated SVD
svd = TruncatedSVD(n_components=100)
X_svd = svd.fit_transform(X_tfidf)


# Étape 5 : Création de variables additionnelles
data['text_length'] = data['cleaned_text'].apply(lambda x: len(x.split()))  # Longueur du texte
data['contains_url'] = data['texte_annonce'].apply(lambda x: 1 if 'http' in x else 0)  # Présence d'URL
data['contains_numbers'] = data['texte_annonce'].apply(lambda x: 1 if re.search(r'\d', x) else 0)  # Présence de chiffres



In [ ]:
# Exporter le DataFrame prétraité en CSV
# Adding the escapechar parameter to handle special characters
data.to_csv("data_preprocessed.csv", index=False, escapechar='\\')

# Ou, pour un fichier binaire plus rapide à charger
data.to_pickle("data_preprocessed.pkl")

In [ ]:
# Afficher le DataFrame transformé
print(data.head())

   OGC_FID boamp_id_annonce boamp_parent_annonce  \
0     9723        23-177335                 None   
1    10502         24-63002                 None   
2    10900         24-94957         ['24-79402']   
3      779        23-179664                 None   
4     5614         24-69326                 None   

                                   boamp_theme_boamp boamp_libelle_annonce  \
0                                      Espaces verts        Avis de marché   
1                      Station d'épuration (travaux)        Avis de marché   
2                           Voirie et réseaux divers          Rectificatif   
3  Prestations de services, Délégation de service...        Avis de marché   
4                                        Génie civil        Avis de marché   

  boamp_statut_annonce boamp_date_fin_de_marche  \
0              INITIAL               2024-02-02   
1              INITIAL               2024-08-08   
2         RECTIFICATIF               2024-08-23   
3             

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd

# Charger le fichier .pkl
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")


# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle de K-NN
knn = KNeighborsClassifier()

# Définir les paramètres à tester
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],   # Nombre de voisins
    'weights': ['uniform', 'distance'],  # Pondération des voisins
    'p': [1, 2]  # Distance de Minkowski : 1 = Manhattan, 2 = Euclidienne
}

# Configurer la recherche par grille
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Exécuter la recherche par grille
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle K-NN optimisé
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec K-NN optimisé:", accuracy)
print("\nClassification Report:\n", report)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:1103: UserWarning: One or more of the test scores are non-finite: [    nan 0.6385  0.6385  0.639       nan 0.7275  0.67775 0.67725     nan
 0.64    0.69625 0.69625     nan 0.7275  0.70775 0.70775     nan 0.72725
 0.71025 0.71025]
  warnings.warn(


Meilleurs paramètres: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
Meilleur score de validation: 0.7275
Accuracy avec K-NN optimisé: 0.741

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.74      1.00      0.85       741
Rejete (hors specs)       0.00      0.00      0.00       259

           accuracy                           0.74      1000
          macro avg       0.37      0.50      0.43      1000
       weighted avg       0.55      0.74      0.63      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Commentaires :

Nous avons commencé par appliquer le modèle des k plus proches voisins. En optimisant ce modèle avec les meilleurs paramètres, on obtient un meilleur modèle avec 5 plus proches voisins, avec la distance de Manhattan et en donnant plus de poids aux plus proches voisins. Cette optimisation permet nous permet d'aboutir à un accuracy score de 0,74. Le modèle détecte très bien les "pris en compte" (taux de précision à 74% Finalement, le modèle des k proches voisins

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assurons-nous que 'X_tfidf' est bien défini
# Exemple (décommenter si nécessaire) :
# tfidf_vectorizer = TfidfVectorizer()
# X_tfidf = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle de régression logistique
log_reg = LogisticRegression(max_iter=1000)

# Définir les paramètres à tester
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],               # Paramètre de régularisation
    'penalty': ['l1', 'l2', 'elasticnet'],      # Type de régularisation
    'solver': ['saga'],                         # Saga fonctionne avec l1, l2 et elasticnet
    'l1_ratio': [0, 0.5, 1]                     # Ratio pour elasticnet, 0 pour l2, 1 pour l1
}

# Configurer la recherche par grille
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Exécuter la recherche par grille
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle de régression logistique optimisé
best_log_reg = grid_search.best_estimator_
y_pred = best_log_reg.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Régression Logistique optimisée:", accuracy)
print("\nClassification Report:\n", report)



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


Meilleurs paramètres: {'C': 10, 'l1_ratio': 0, 'penalty': 'l2', 'solver': 'saga'}
Meilleur score de validation: 0.745
Accuracy avec Régression Logistique optimisée: 0.773

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.79      0.94      0.86       741
Rejete (hors specs)       0.64      0.29      0.39       259

           accuracy                           0.77      1000
          macro avg       0.71      0.61      0.63      1000
       weighted avg       0.75      0.77      0.74      1000



In [ ]:
# Initialiser le modèle de régression logistique avec les meilleurs paramètres
best_log_reg = LogisticRegression(C=10, l1_ratio=0, penalty='l2', solver='saga', max_iter=1000)

# Entraîner le modèle avec les données d'entraînement
best_log_reg.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = best_log_reg.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Régression Logistique optimisée:", accuracy)
print("\nClassification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


Accuracy avec Régression Logistique optimisée: 0.77

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.77      0.98      0.86       741
Rejete (hors specs)       0.74      0.17      0.28       259

           accuracy                           0.77      1000
          macro avg       0.75      0.58      0.57      1000
       weighted avg       0.76      0.77      0.71      1000



In [ ]:
# cal_réponse_signalement est notre variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle d'arbre de décision
tree = DecisionTreeClassifier(random_state=42)

# Définir les paramètres à tester
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Configurer et exécuter la recherche par grille
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle optimisé avec les meilleurs paramètres
best_tree = grid_search.best_estimator_
y_pred = best_tree.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Arbre de Décision optimisé:", accuracy)
print("\nClassification Report:\n", report)

Meilleurs paramètres: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}
Meilleur score de validation: 0.7220000000000001
Accuracy avec Arbre de Décision optimisé: 0.76

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.77      0.96      0.86       741
Rejete (hors specs)       0.61      0.20      0.30       259

           accuracy                           0.76      1000
          macro avg       0.69      0.58      0.58      1000
       weighted avg       0.73      0.76      0.71      1000



In [ ]:
# prompt: Meilleurs paramètres: {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 10}

import pandas as pd
import nltk
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


# Initialiser le modèle d'arbre de décision avec les meilleurs paramètres
best_tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, min_samples_split=10, random_state=42)

# Entraîner le modèle avec les données d'entraînement
best_tree.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = best_tree.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Arbre de Décision optimisé (avec meilleurs paramètres):", accuracy)
print("\nClassification Report:\n", report)

Accuracy avec Arbre de Décision optimisé (avec meilleurs paramètres): 0.749

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.77      0.94      0.85       741
Rejete (hors specs)       0.54      0.21      0.30       259

           accuracy                           0.75      1000
          macro avg       0.66      0.57      0.58      1000
       weighted avg       0.71      0.75      0.71      1000



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser et entraîner la forêt aléatoire
forest = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators est le nombre d'arbres
forest.fit(X_train, y_train)

# Prédire les labels sur l'ensemble de test
y_pred = forest.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Forêt Aléatoire:", accuracy)
print("\nClassification Report:\n", report)

Accuracy avec Forêt Aléatoire: 0.743

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.75      0.99      0.85       741
Rejete (hors specs)       0.57      0.03      0.06       259

           accuracy                           0.74      1000
          macro avg       0.66      0.51      0.45      1000
       weighted avg       0.70      0.74      0.65      1000



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Charger le fichier .pkl
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")


# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialiser le modèle de forêt aléatoire
forest = RandomForestClassifier(random_state=42)

# Définir les paramètres à tester pour l'optimisation
param_grid = {
    'n_estimators': [50, 100, 200],           # Nombre d'arbres dans la forêt
    'max_depth': [10, 20, None],              # Profondeur maximale de chaque arbre
    'min_samples_split': [2, 5, 10],          # Nombre minimum d'échantillons pour diviser un nœud
    'min_samples_leaf': [1, 2, 4]             # Nombre minimum d'échantillons par feuille
}

# Configurer et exécuter la recherche par grille
grid_search = GridSearchCV(forest, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et score
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleur score de validation:", grid_search.best_score_)

# Entraîner le modèle optimisé avec les meilleurs paramètres
best_forest = grid_search.best_estimator_
y_pred = best_forest.predict(X_test)

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Forêt Aléatoire optimisée:", accuracy)
print("\nClassification Report:\n", report)

Meilleurs paramètres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Meilleur score de validation: 0.728
Accuracy avec Forêt Aléatoire optimisée: 0.745

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.75      0.99      0.85       741
Rejete (hors specs)       0.62      0.04      0.07       259

           accuracy                           0.74      1000
          macro avg       0.69      0.52      0.46      1000
       weighted avg       0.72      0.74      0.65      1000



In [ ]:
# prompt: Meilleurs paramètres: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

# Initialiser le modèle de forêt aléatoire avec les meilleurs paramètres
best_forest = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42)

# Entraîner le modèle avec les données d'entraînement
best_forest.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = best_forest.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy avec Forêt Aléatoire optimisée (avec meilleurs paramètres):", accuracy)
print("\nClassification Report:\n", report)

Accuracy avec Forêt Aléatoire optimisée (avec meilleurs paramètres): 0.748

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.75      1.00      0.85       741
Rejete (hors specs)       0.82      0.03      0.07       259

           accuracy                           0.75      1000
          macro avg       0.78      0.52      0.46      1000
       weighted avg       0.77      0.75      0.65      1000



In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Charger les données prétraitées d'entraînement
data_preprocessed = pd.read_pickle("/content/data_preprocessed.pkl")

# Définir la variable cible
y_train = data_preprocessed['cal_réponse_signalement']

# Créer la matrice TF-IDF pour les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Initialiser et entraîner le modèle de régression logistique avec les paramètres optimisés
log_reg = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=1000)
log_reg.fit(X_train, y_train)

# Charger et prétraiter les données de test
test_data = pd.read_json("/content/test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test en utilisant le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Effectuer les prédictions sur les données de test
y_pred = log_reg.predict(X_test)

# Ajouter les prédictions au DataFrame de test pour référence
test_data['predictions'] = y_pred

# Exporter les prédictions pour les données de test dans un fichier CSV
test_data[['texte_annonce', 'predictions']].to_csv("test_predictions.csv", index=False)

print("Les prédictions ont été enregistrées dans le fichier 'test_predictions.csv'.")


Les prédictions ont été enregistrées dans le fichier 'test_predictions.csv'.


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger les données d'entraînement prétraitées
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Charger et prétraiter les données de test
test_data = pd.read_json("test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test avec le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Initialiser et entraîner le modèle de régression logistique
log_reg = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=1000)
log_reg.fit(X_train, y_train)

# Prédire les valeurs pour cal_réponse_signalement
y_pred = log_reg.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_pred

# Créer le fichier final avec uniquement les colonnes OCG_FID et cal_réponse_signalement
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]

# Exporter le fichier final en CSV
final_output.to_csv("final_test_predictions.csv", index=False)

print("Le fichier 'final_test_predictions.csv' a été généré avec succès.")



Le fichier 'final_test_predictions.csv' a été généré avec succès.


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger les données d'entraînement prétraitées
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Charger et prétraiter les données de test
test_data = pd.read_json("test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test avec le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Initialiser le modèle de forêt aléatoire avec les meilleurs paramètres
best_forest = RandomForestClassifier(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42)

# Entraîner le modèle avec les données d'entraînement
best_forest.fit(X_train, y_train)

# Prédire les valeurs pour cal_réponse_signalement
y_pred = best_forest.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_pred

# Créer le fichier final avec uniquement les colonnes OGC_FID et cal_réponse_signalement
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]

# Exporter le fichier final en CSV
final_output.to_csv("final_test_predictions_forest.csv", index=False)

print("Le fichier 'final_test_predictions_forest.csv' a été généré avec succès.")


Le fichier 'final_test_predictions_forest.csv' a été généré avec succès.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split

# --- Étape 1 : Charger les données d'entraînement prétraitées ---
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les textes d'entraînement
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.8)
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# --- Étape 2 : Charger les données de test ---
test_data = pd.read_json("test.jsonl", lines=True)

# Nettoyer les textes de test
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Supprimer les caractères spéciaux et chiffres
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test_data['cleaned_text'] = test_data['texte_annonce'].apply(clean_text)

# Transformer les textes de test avec le même TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# --- Étape 3 : Initialiser le modèle SVM ---
# Définir un SVM avec recherche d'hyperparamètres
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto', 0.01, 0.1],
    'kernel': ['linear', 'rbf']
}

svm = SVC(probability=True, random_state=42)

# Recherche des meilleurs hyperparamètres
grid_search = GridSearchCV(svm, param_grid, scoring='f1_weighted', cv=5)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres
print("Meilleurs paramètres :", grid_search.best_params_)
best_svm = grid_search.best_estimator_

# --- Étape 4 : Prédictions sur les données de test ---
y_test_pred = best_svm.predict(X_test)
y_test_probs = best_svm.predict_proba(X_test)[:, 1]  # Probabilités de la classe positive

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred
test_data['confidence'] = y_test_probs

# --- Étape 5 : Exporter les prédictions ---
test_data[['OGC_FID', 'cal_réponse_signalement_prédite', 'confidence']].to_csv("svm_test_predictions.csv", index=False)
print("Les prédictions du modèle SVM ont été exportées dans 'svm_test_predictions.csv'.")

# --- Étape 6 : Évaluer le modèle sur des données avec des étiquettes (si disponibles) ---
# Supposons que vous avez les vraies étiquettes dans une colonne 'cal_réponse_signalement'
if 'cal_réponse_signalement' in test_data.columns:
    print("\nÉvaluation des performances du SVM :")
    print(classification_report(test_data['cal_réponse_signalement'], y_test_pred))
    print(f"F1-Score : {f1_score(test_data['cal_réponse_signalement'], y_test_pred, pos_label='Pris en compte', average='weighted'):.3f}")


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score
import re  # Ajouter cette ligne au début de votre code


# --- Étape 1 : Charger les données d'entraînement prétraitées ---
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les textes d'entraînement
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5, max_df=0.8)
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# --- Étape 2 : Charger les données de test ---
test_data = pd.read_json("test.jsonl", lines=True)

# Nettoyer les textes de test
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Supprimer les caractères spéciaux et chiffres
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test_data['cleaned_text'] = test_data['texte_annonce'].apply(clean_text)

# Transformer les textes de test avec le même TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# --- Étape 3 : Initialiser et entraîner le modèle SVM ---
# Utilisation de paramètres fixes pour un essai rapide
svm = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)
svm.fit(X_train, y_train)

# --- Étape 4 : Faire des prédictions sur les données de test ---
y_test_pred = svm.predict(X_test)
y_test_probs = svm.predict_proba(X_test)[:, 1]  # Probabilités de la classe positive

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred
test_data['confidence'] = y_test_probs

# --- Étape 5 : Exporter les prédictions ---
test_data[['OGC_FID', 'cal_réponse_signalement_prédite']].to_csv("svm_test_predictions.csv", index=False)
print("Les prédictions du modèle SVM ont été exportées dans 'svm_test_predictions.csv'.")

# --- Étape 6 : Évaluer le modèle sur des données avec des étiquettes (si disponibles) ---
# Supposons que vous avez les vraies étiquettes dans une colonne 'cal_réponse_signalement'
if 'cal_réponse_signalement' in test_data.columns:
    print("\nÉvaluation des performances du SVM :")
    print(classification_report(test_data['cal_réponse_signalement'], y_test_pred))
    print(f"F1-Score : {f1_score(test_data['cal_réponse_signalement'], y_test_pred, pos_label='Pris en compte', average='weighted'):.3f}")


Les prédictions du modèle SVM ont été exportées dans 'svm_test_predictions.csv'.


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score

# Charger les données d'entraînement
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Pipeline pour inclure la vectorisation TF-IDF et le modèle SVM
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', SVC(probability=True, random_state=42))
])

# Définir les grilles de paramètres à tester
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # Uni-gramme, bi-gramme
    'tfidf__min_df': [1, 5, 10],             # Fréquence min des termes
    'tfidf__max_df': [0.5, 0.8, 1.0],        # Fréquence max des termes
    'svm__C': [0.1, 1, 10],                  # Régularisation
    'svm__kernel': ['linear', 'rbf'],        # Type de noyau
    'svm__gamma': ['scale', 'auto']          # Coefficient du noyau
}

# Initialiser GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    scoring='f1_weighted',  # Utiliser le F1-Score pondéré comme métrique
    cv=3,                   # Validation croisée à 3 plis
    verbose=3,              # Afficher les logs
    n_jobs=-1               # Utiliser tous les cœurs disponibles
)

# Lancer l'optimisation
grid_search.fit(data_preprocessed['cleaned_text'], y_train)

# Afficher les meilleurs paramètres et le score associé
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score F1 pondéré :", grid_search.best_score_)

# Sauvegarder le modèle optimisé
best_svm = grid_search.best_estimator_


Fitting 3 folds for each of 216 candidates, totalling 648 fits


KeyboardInterrupt: 

#Optimisation

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import pandas as pd
import numpy as np

# Charger le fichier .pkl
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")

# Définir la variable cible
y = data_preprocessed['cal_réponse_signalement']

# Supprimer les colonnes inutiles (par exemple, identifiants ou colonnes textuelles longues)
columns_to_drop = ['OGC_FID', 'texte_annonce']  # Ajuster selon vos données
X = data_preprocessed.drop(columns=columns_to_drop + ['cal_réponse_signalement'], errors='ignore')

# Identifier les colonnes qualitatives restantes et les encoder si nécessaire
from sklearn.preprocessing import LabelEncoder

non_numeric_columns = X.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    X[col] = label_encoder.fit_transform(X[col])

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Étape 1 : Réduction des variables avec RandomForestClassifier ---
# Entraîner un modèle de forêt aléatoire pour évaluer l'importance des variables
forest = RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)

# Importance des variables
importances = forest.feature_importances_
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances}).sort_values(by='Importance', ascending=False)
print("\nTop 10 variables importantes :\n", importance_df.head(10))

# Réduire les variables en ne gardant que celles au-dessus d'un seuil d'importance
threshold = 0.01
important_features = importance_df[importance_df['Importance'] > threshold]['Feature'].tolist()
X_train_reduced = X_train[important_features]
X_test_reduced = X_test[important_features]

# Vérifier les dimensions après réduction
print(f"\nNombre de colonnes après réduction : {X_train_reduced.shape[1]}")



Top 10 variables importantes :
                             Feature  Importance
11                   boamp_intitule    0.065873
2                 boamp_theme_boamp    0.057482
7                boamp_nom_acheteur    0.054459
22           cal_insee_commune_exec    0.054423
25                     cleaned_text    0.052405
16                   cal_mot_rang_1    0.052240
17                   cal_mot_rang_2    0.051991
6   boamp_num_departement_diffusion    0.051126
26                      text_length    0.050604
23             cal_nom_commune_exec    0.050174

Nombre de colonnes après réduction : 23


In [ ]:
# Réentraîner le modèle avec les données originales et gestion des poids
forest_weighted = RandomForestClassifier(
    max_depth=None,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=50,
    class_weight='balanced',  # Gestion automatique des poids
    random_state=42
)

# Entraîner le modèle
forest_weighted.fit(X_train_reduced, y_train)

# Faire des prédictions sur les données de test
y_pred_weighted = forest_weighted.predict(X_test_reduced)

# Évaluer les performances
accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
report_weighted = classification_report(y_test, y_pred_weighted)

print("\n--- Résultats avec Forêt Aléatoire et Gestion des Poids ---")
print("Accuracy :", accuracy_weighted)
print("\nClassification Report:\n", report_weighted)




--- Résultats avec Forêt Aléatoire et Gestion des Poids ---
Accuracy : 0.753

Classification Report:
                      precision    recall  f1-score   support

     Pris en compte       0.76      0.98      0.85       741
Rejete (hors specs)       0.63      0.11      0.19       259

           accuracy                           0.75      1000
          macro avg       0.69      0.54      0.52      1000
       weighted avg       0.73      0.75      0.68      1000



In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

# Charger les données d'entraînement prétraitées
data_preprocessed = pd.read_pickle("data_preprocessed.pkl")
y_train = data_preprocessed['cal_réponse_signalement']

# Vectoriser les données d'entraînement
tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(data_preprocessed['cleaned_text'])

# Charger et prétraiter les données de test
test_data = pd.read_json("test.jsonl", lines=True)
test_data['cleaned_text'] = test_data['texte_annonce'].str.lower().str.replace(r'[^a-z\s]', '', regex=True)

# Transformer les données de test avec le même vectoriseur TF-IDF
X_test = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Initialiser et entraîner le modèle de régression logistique
log_reg = LogisticRegression(C=10, penalty='l2', solver='saga', max_iter=1000)
log_reg.fit(X_train, y_train)

# Prédire les valeurs pour cal_réponse_signalement
y_pred = log_reg.predict(X_test)

# Ajouter les prédictions dans le DataFrame de test
test_data['cal_réponse_signalement'] = y_pred

# Créer le fichier final avec uniquement les colonnes OCG_FID et cal_réponse_signalement
final_output = test_data[['OGC_FID', 'cal_réponse_signalement']]

# Exporter le fichier final en CSV
final_output.to_csv("final_test_predictions.csv", index=False)

print("Le fichier 'final_test_predictions.csv' a été généré avec succès.")



Le fichier 'final_test_predictions.csv' a été généré avec succès.


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Charger les données de test avec la vérité terrain
test_data = pd.read_json("test.jsonl", lines=True)
# Charger les prédictions finales
test_predictions = pd.read_csv("/content/final_test_predictions.csv")
# Associer les prédictions aux vérités terrain en utilisant la colonne `OGC_FID`
merged_data = test_data.merge(test_predictions, on='OGC_FID', suffixes=('_réelle', '_prédite'))

# Vérifier si les colonnes nécessaires existent
if 'cal_réponse_signalement' in merged_data.columns and 'cal_réponse_signalement_prédite' in merged_data.columns:
    # Calculer l'accuracy
    accuracy = accuracy_score(merged_data['cal_réponse_signalement'], merged_data['cal_réponse_signalement_prédite'])

    # Générer le rapport de classification
    report = classification_report(merged_data['cal_réponse_signalement'], merged_data['cal_réponse_signalement_prédite'])

    # Matrice de confusion
    cm = confusion_matrix(merged_data['cal_réponse_signalement'], merged_data['cal_réponse_signalement_prédite'])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Rejeté", "Pris en compte"])

    print(f"Accuracy : {accuracy:.3f}\n")
    print("Rapport de classification :\n", report)

    # Afficher la matrice de confusion
    disp.plot(cmap="Blues")
    plt.title("Matrice de Confusion : Modèle Final")
    plt.show()
else:
    print("Les colonnes nécessaires (prédictions ou vérités terrain) ne sont pas disponibles.")


Les colonnes nécessaires (prédictions ou vérités terrain) ne sont pas disponibles.


In [ ]:
important_features = [
    'boamp_intitule', 'boamp_theme_boamp', 'boamp_nom_acheteur',
    'cal_insee_commune_exec', 'cleaned_text', 'cal_mot_rang_1',
    'cal_mot_rang_2', 'boamp_num_departement_diffusion', 'text_length',
    'cal_nom_commune_exec'
]
# Les colonnes utilisées dans votre DataFrame d'entraînement
top_29_features = X_train.columns.tolist()

# Vérifiez le contenu de la liste
print("Liste des 29 features utilisées dans le modèle :")
print(top_29_features)
print(f"Nombre de features : {len(top_29_features)}")


Liste des 29 features utilisées dans le modèle :
['boamp_id_annonce', 'boamp_parent_annonce', 'boamp_theme_boamp', 'boamp_libelle_annonce', 'boamp_statut_annonce', 'boamp_date_fin_de_marche', 'boamp_num_departement_diffusion', 'boamp_nom_acheteur', 'boamp_siret_acheteur', 'boamp_type_organisme', 'boamp_lieu_exec', 'boamp_intitule', 'cal_fichier_pdf', 'cal_num_signalement', 'cal_theme_signalement', 'cal_statut_workflow', 'cal_mot_rang_1', 'cal_mot_rang_2', 'cal_date_traitement', 'cal_type_localisation', 'cal_num_departement_exec', 'cal_siren_epci', 'cal_insee_commune_exec', 'cal_nom_commune_exec', 'cal_adresse_annonce', 'cleaned_text', 'text_length', 'contains_url', 'contains_numbers']
Nombre de features : 29


In [ ]:
# Étape 1 : Charger les données de test
test_data = pd.read_json("test.jsonl", lines=True)

# Étape 2 : Nettoyage et lemmatisation du texte
test_data['cleaned_text'] = test_data['texte_annonce'].apply(clean_text).apply(lemmatize_text)

# Étape 3 : Vectorisation avec le TF-IDF déjà entraîné
# Important : Utiliser le vectoriseur TF-IDF ajusté sur les données d'entraînement
X_test_tfidf = tfidf_vectorizer.transform(test_data['cleaned_text'])

# Étape 4 : Réduction de dimension (SVD)
# Important : Utiliser le modèle SVD déjà entraîné
X_test_reduced = svd.transform(X_test_tfidf)

# Étape 5 : Création des variables additionnelles
test_data['text_length'] = test_data['cleaned_text'].apply(lambda x: len(x.split()))
test_data['contains_url'] = test_data['texte_annonce'].apply(lambda x: 1 if 'http' in x else 0)
test_data['contains_numbers'] = test_data['texte_annonce'].apply(lambda x: 1 if re.search(r'\d', x) else 0)

# Étape 6 : Sélection des colonnes additionnelles et conversion en array
additional_features = test_data[['text_length', 'contains_url', 'contains_numbers']].values

# Étape 7 : Combinaison des variables réduites et additionnelles
X_test_combined = np.hstack((X_test_reduced, additional_features))

# Étape 8 : Sélection des 29 features importants
# Important : Utiliser les mêmes features que ceux sélectionnés pour l'entraînement
selected_features = top_29_features
X_test_final = test_data[selected_features].values



In [ ]:
# Vérification des dimensions
if X_test_final.shape[1] != X_train.shape[1]:
    raise ValueError(f"Incompatibilité : X_test_final a {X_test_final.shape[1]} features, mais le modèle en attend {X_train.shape[1]}.")


In [ ]:
missing_features = [col for col in important_features if col not in test_data.columns]
if missing_features:
    print(f"Colonnes manquantes dans les données de test : {missing_features}")
else:
    print("Toutes les colonnes importantes sont disponibles.")


Toutes les colonnes importantes sont disponibles.


In [ ]:
# Étape 9 : Faire des prédictions avec le modèle optimisé
y_test_pred = best_forest.predict(X_test_final)
y_test_probs = best_forest.predict_proba(X_test_final)[:, 1]

# Étape 10 : Ajouter les prédictions et scores dans le DataFrame de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred
test_data['confidence'] = y_test_probs

# Étape 11 : Exporter les prédictions pour analyse
test_data[['OGC_FID', 'cal_réponse_signalement_prédite']].to_csv("test_predictions.csv", index=False)

print("Les prédictions ont été exportées dans 'test_predictions.csv'.")


NameError: name 'best_forest' is not defined

In [ ]:
if X_test_final.shape[1] != X_train.shape[1]:
    print(f"Incompatibilité : X_test_final a {X_test_final.shape[1]} features, mais le modèle en attend {X_train.shape[1]}.")


Incompatibilité : X_test_final a 103 features, mais le modèle en attend 29.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'train_data' is your training data DataFrame
# ... (your previous code to load and preprocess train_data) ...

# 1. Re-fit TF-IDF Vectorizer on the combined text data of train and test sets
all_text_data = pd.concat([train_data['cleaned_text'], test_data['cleaned_text']])
tfidf_vectorizer = TfidfVectorizer()  # Or use your previous parameters
tfidf_vectorizer.fit(all_text_data)

# 2. Transform train and test data with the re-fitted TF-IDF Vectorizer
X_train_tfidf = tfidf_vectorizer.transform(train_data['cleaned_text'])
X_test_tfidf = tfidf_vectorizer.transform(test_data['cleaned_text'])

# 3. Re-fit SVD on the transformed training data
svd.fit(X_train_tfidf)  # Or use your previous parameters

# 4. Transform both train and test data with the re-fitted SVD
X_train_reduced = svd.transform(X_train_tfidf)
X_test_reduced = svd.transform(X_test_tfidf)

# 5. Continue with creating and combining additional features as before
# ... (your existing code for additional features and combining them) ...

# 6. Retrain your model (best_forest) with the new transformed training data
# ... (your existing code for model training) ...

# 7. Now predict using the updated test data
y_test_pred = best_forest.predict(X_test_final)
# ... (rest of your prediction code) ...

NameError: name 'train_data' is not defined

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Étape 9 : Analyse des résultats (si étiquettes disponibles dans test_data)
if 'cal_réponse_signalement' in test_data.columns:
    print("\n--- Analyse des Résultats ---")
    print("Classification Report:")
    print(classification_report(test_data['cal_réponse_signalement'], y_test_pred))

    # Matrice de confusion
    cm = confusion_matrix(test_data['cal_réponse_signalement'], y_test_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Rejeté", "Pris en compte"])
    disp.plot(cmap="Blues")
    plt.title("Matrice de Confusion : Forêt Aléatoire Optimisée")
    plt.show()

    # Afficher l'Accuracy
    accuracy = accuracy_score(test_data['cal_réponse_signalement'], y_test_pred)
    print(f"\nAccuracy sur les données de test : {accuracy:.3f}")

# Étape 10 : Exporter les prédictions pour analyse
test_data[['OGC_FID', 'cal_réponse_signalement_prédite', 'confidence']].to_csv("test_predictions.csv", index=False)
print("\nLes prédictions ont été exportées dans 'test_predictions.csv'.")


In [ ]:
# Faire des prédictions avec le modèle optimisé
y_test_pred = best_forest.predict(X_test_final)
y_test_probs = best_forest.predict_proba(X_test_final)[:, 1]

# Ajouter les prédictions et scores dans le DataFrame de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred
test_data['confidence'] = y_test_probs

# Analyse des résultats
print("\n--- Analyse des Résultats ---")
if 'cal_réponse_signalement' in test_data.columns:  # Si les vraies étiquettes sont disponibles
    print("Classification Report:")
    print(classification_report(test_data['cal_réponse_signalement'], y_test_pred))

    # Afficher la matrice de confusion
    cm = confusion_matrix(test_data['cal_réponse_signalement'], y_test_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Rejeté", "Pris en compte"])
    disp.plot(cmap="Blues")
    plt.title("Matrice de Confusion : Forêt Aléatoire Optimisée")
    plt.show()

    # Afficher l'Accuracy
    accuracy = accuracy_score(test_data['cal_réponse_signalement'], y_test_pred)
    print(f"\nAccuracy sur les données de test : {accuracy:.3f}")

# Exporter les prédictions pour analyse
test_data[['OGC_FID', 'cal_réponse_signalement_prédite', 'confidence']].to_csv("test_predictions.csv", index=False)
print("Les prédictions ont été exportées dans 'test_predictions.csv'.")



ValueError: could not convert string to float: "Travaux de création d'une piste cyclable située rue des Poissonniers/Cantelaude/Moulin à Le Teich"

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Charger les données de test
test_data = pd.read_json("test.jsonl", lines=True)

# Réduction des variables sur les données de test
X_test_reduced = test_data[important_features]  # Utilisez les variables importantes identifiées

# Prédire les classes sur les données de test avec le modèle pondéré
y_test_pred_weighted = forest_weighted.predict(X_test_reduced)

# Ajouter les prédictions dans le DataFrame des données de test
test_data['cal_réponse_signalement_prédite'] = y_test_pred_weighted

# Exporter les prédictions
test_data[['OGC_FID', 'cal_réponse_signalement_prédite']].to_csv("test_predictions_weighted.csv", index=False)

print("\nLes prédictions ont été enregistrées dans 'test_predictions_weighted.csv'.")


KeyError: "['cleaned_text', 'text_length'] not in index"